# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
...,...,...,...,...,...,...,...,...,...,...
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564
574,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565
575,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,1666108566


In [20]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', "Lat", geo = True, tiles = "OSM", size="Humidity", scale=0.5, color="City")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data
reduced_list = city_data_df[(city_data_df['Max Temp']>15.0) & (city_data_df['Max Temp']<20.5) & 
    (city_data_df['Cloudiness']<10) & (city_data_df['Wind Speed']<5)]
drop_na = reduced_list.dropna()
reduced_list

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,margate,51.3813,1.3862,16.84,76,3,1.34,GB,1666108256
128,128,half moon bay,37.4636,-122.4286,19.71,86,0,0.00,US,1666108312
276,276,tura,25.5198,90.2201,20.45,90,1,1.51,IN,1666108399
303,303,alice springs,-23.7000,133.8833,15.75,63,0,4.63,AU,1666108416
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417
314,314,horsham,51.0640,-0.3272,18.40,72,0,4.12,GB,1666108422
318,318,harwich,51.9419,1.2844,17.38,69,1,3.13,GB,1666108424
451,451,san joaquin,37.9333,-121.3011,20.30,89,0,2.57,US,1666108498
488,488,pleasanton,37.6624,-121.8747,18.84,86,0,0.00,US,1666108325
510,510,wadi musa,30.3222,35.4793,18.54,63,4,1.15,JO,1666108534


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data
hotel_df = reduced_list[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df["Hotel Name"] = ''
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
42,margate,GB,51.3813,1.3862,76,
128,half moon bay,US,37.4636,-122.4286,86,
276,tura,IN,25.5198,90.2201,90,
303,alice springs,AU,-23.7000,133.8833,63,
305,kingman,US,35.1894,-114.0530,49,
314,horsham,GB,51.0640,-0.3272,72,
318,harwich,GB,51.9419,1.2844,69,
451,san joaquin,US,37.9333,-121.3011,89,
488,pleasanton,US,37.6624,-121.8747,86,
510,wadi musa,JO,30.3222,35.4793,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
latitude = []
longitude = []
params = {'apiKey':geoapify_key, 'categories': "accommodation.hotel"}

# Print a message to follow up the hotel search
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    filters = [f'circle:{longitude},{latitude},{radius}']
    bias = [f'proximity:{longitude},{latitude}']
    params['categories'] = f'accommodation.hotel'
    params['filter'] = filters
    params['bias'] = bias
        
# Set base URL
    base_url = "https://api.geoapify.com/v2/places"

# Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    #data = name_address.json()

# Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
# Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margate - nearest hotel: The Reading Rooms
half moon bay - nearest hotel: Half Moon Bay Inn
tura - nearest hotel: Hotel Polo Orchid
alice springs - nearest hotel: Aurora Alice Springs
kingman - nearest hotel: Home2 Suites by Hilton Kingman
horsham - nearest hotel: Premier Inn
harwich - nearest hotel: The Bottle Kiln
san joaquin - nearest hotel: Days Inn Stockton
pleasanton - nearest hotel: The Rose Hotel
wadi musa - nearest hotel: Elgee Hotel
gifhorn - nearest hotel: Alte Münze
lompoc - nearest hotel: Red Roof Inn Lompoc
iquique - nearest hotel: Hostal Vivar


,City,Country,Lat,Lng,Humidity,Hotel Name
42,margate,GB,51.3813,1.3862,76,The Reading Rooms
128,half moon bay,US,37.4636,-122.4286,86,Half Moon Bay Inn
276,tura,IN,25.5198,90.2201,90,Hotel Polo Orchid
303,alice springs,AU,-23.7000,133.8833,63,Aurora Alice Springs
305,kingman,US,35.1894,-114.0530,49,Home2 Suites by Hilton Kingman
314,horsham,GB,51.0640,-0.3272,72,Premier Inn
318,harwich,GB,51.9419,1.2844,69,The Bottle Kiln
451,san joaquin,US,37.9333,-121.3011,89,Days Inn Stockton
488,pleasanton,US,37.6624,-121.8747,86,The Rose Hotel
510,wadi musa,JO,30.3222,35.4793,63,Elgee Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
hotel_df.dtypes

City           object
Country        object
Lat           float64
Lng           float64
Humidity        int64
Hotel Name     object
dtype: object

In [31]:
%%capture --no-display
#%%capture --no-display
#This is in 09-live banking from 10/7/2024 class
# Configure the map plot
city_hotel_name_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo = True,
    tiles = 'OSM', 
    size = 'Humidity', 
    scale = 1,
    color = 'Hotel Name',
    hover_cols = ['Hotel Name', 'Country'],
    frame_width = 800, 
    frame_height = 600
)

# Display the map
city_hotel_name_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)